In [117]:
import pandas as pd
import recordlinkage
pd.options.display.max_columns = None

### Load Data fiels

In [428]:
ama = pd.read_stata('ama13.dta')

In [429]:
meetings = pd.read_excel('speakerData.xlsx')
meetings = meetings.drop("meetingID", axis='columns')
meetings['nonus'] = ~(meetings.us==1)

In [431]:
medicareHospitals = pd.read_excel('medicareAcademicHospitals.xlsx')

In [432]:
medicareHospitals.head(1)

,TIN,Hospital Name,PECOS Legal Business Name,Street Address,PO Box,City,State,Zip Code,Street Address 1,Street Address 2,City.1,State.1,Zip Code.1
0,920016429,PROVIDENCE ALASKA MEDICAL CENTER,Providence Health & Services - Washington,3200 PROVIDENCE DRIVE,PO BOX 196604,ANCHORAGE,AK,99519,3200 Providence Dr,NaN,Anchorage,AK,99508


In [167]:
ama.loc[ama.npi=="1528185246"]

,ResearchID,npi,MailFirstName,MailMiddleName,MailLastName,AddressType,MailAddress1,MailZipCode,MailZip5,Undeliverable,FIPSCounty,FIPSCity,FIPSState,CensusRegion,CensusDivision,CensusGroup,CensusTract,CensusSuffix,CensusBlockGroup,CensusEnumerationGroup,cmsa,PMSA_MSA,smsa,msa,DOFlag,BirthDate,BirthCity,BirthState,BirthCountry,Sex,DeadFlag,LicPrefState,LicPrefYear,LicGrpState,LicGrpYear,LicHospState,LicHospYear,LicAltOffState,LicAltOffYear,PrimaryTOP,PresentEmployment,PrimarySpecialty,SecondarySpecialty,tops,HospitalID,HospitalHours,PRAFlag,PRAExpiration,GroupID1,GroupID2,MedTrainFlag,MedTrainFrom,MedTrainTo,MedTrainYearInProgram,MedTrainPostGradYear,MedTrainSpec1,MedTrainSpec2,MedTrainingInstitutionCode,MedSchoolID,MedSchoolYOG,ContactFlag,NumberOfOffices,OfficeState,OfficeZipCode,USTrained,ResidencyTrainingState,MedicalSchoolState,OmitFlag,yob,yot
559538,3740050891,1528185246,James,Francis,Burke,2,NA,48103-3147,48103,0,161,NA,26,2.0,3.0,6,4007,00,0,NA,35,0440,NA,A,0,5844.0,CLEVELAND,OH,US1,M,0,MI,2009,NA,NA,NA,NA,MI,2009,020,110,VN,US,PO,NA,000,0,NaN,NA,NA,0,NA,18443.0,0,0,NA,NA,250256,01643,2005,1,0,MI,48109-5002,0,MI,IL,NA,1976,2010.0


### Step 1: ...assign hospoitals to zip codes from the ama hospital list

In [ ]:
hospitalMatchIndex = recordlinkage.index.Full()
#hospitalMatchIndex = recordlinkage.index.SortedNeighbourhood(left_on='homeInstitution', right_on='Hospital Name', window=1000)
hospMatchIndex = hospitalMatchIndex.index(meetings, medicareHospitals)
hospMatchIndex.names = ['meeting', 'medicare']
dfTest = pd.DataFrame(index=hospMatchIndex)
dfTest.sort_index().head(5)

In [ ]:
comp = recordlinkage.Compare(n_jobs=12)
comp.string('homeInstitution', 'Hospital Name', method='levenshtein')
compared = comp.compute(hospMatchIndex, meetings, medicareHospitals)
compared = compared.rename({0:"weight"}, axis='columns')

In [ ]:
compared.sort_values(['meeting', 'weight'], ascending=False, inplace=True)

In [ ]:
compared.head()

Summary: Matching on the hospital names that we have is pretty awful...not obvious that its going to do anything at all for us. Will probably need to go back adn pull out zip codes for our poeple if we'r egoing ot make this work. I tried a couple algorthims and a copule strategies...but, the names are just very different. And sometimes the health center name and hospital name compeltely diverge. I don't think ths is going to be fruitful.

### Step 2: filter ama list to neurology and related specialities.

Neurology specialities from AMA masterfile

<li>MN	Internal Medicine - Neurology</li>
<li>N	Neurology</li>
<li>NDN	Neurodevelopmental Disabilities</li>
<li>NDP	Neurodevelopmental Disabilities</li>
<li>NMN	Neuromuscliar Medicine</li>
<li>NNM	Neurology / Nuclear Medicine</li>
<li>NS	Neurological Surgery</li>
<li>NSP	Pediatric Neurological Surgery</li>
<li>NUP	Neuropsychiatry</li>
<li>PMN	Pain Medicine (Neurology)</li>
<li>PYN	Psychiatry - Neurology</li>
<li>SMN	Sleep Medicine (Psych & Neurology)</li>
<li>VN	Vascliar Neurology</li>
<li>CHN Child neurology</li>

In [433]:
ama['MailFirstName'] = ama['MailFirstName'].str.lower()
ama['MailLastName'] = ama['MailLastName'].str.lower()
ama['MailMiddleName'] = ama['MailMiddleName'].str.lower()
meetings['firstName'] = meetings['firstName'].str.lower()
meetings['lastName'] = meetings['lastName'].str.lower()
ama['middleInitial'] = ama['MailMiddleName'].str[:1]
# new data frame with split value columns 
splitName = meetings["firstName"].str.split(" ", n = 1, expand = True) 
# making seperate first name column from new data frame 
meetings["firstName"]= splitName[0] 
meetings["middleInitial"]= splitName[1].str.strip(".")
meetings["middleInitial"]= meetings['middleInitial'].str[:1]
ama['female'] = ama['Sex'] == 'F'
ama['female'] = ama['female'].astype(int)
# the Year of graduation field looks awful...
ama['MedSchoolYOG'] = ama['MedSchoolYOG'].astype(int)
ama['YOGEst'] = ama.yot - 4

In [434]:
neuroSpecialties = ['MN', 'N', 'NDN', 'NMN', 'NNM', 'NS', 'NSP', 'NUP', 'PMN', 'PYN', 'SMN', 'VN', 'CHN']
primaryNeuro = ama.loc[ama['PrimarySpecialty'].isin(neuroSpecialties)]
secondaryNeuro = ama.loc[ama['SecondarySpecialty'].isin(neuroSpecialties)]
allNeuro = pd.concat([primaryNeuro,secondaryNeuro]).drop_duplicates().reset_index(drop=True)

In [435]:
meetings['totalNonMissing'] = meetings[['lastName', 'firstName', 'middleInitial', 'female', 'medGradYear']].notnull().sum(axis=1)
meetings['weightedNonMissing'] = meetings['middleInitial'].notnull() * 0.25 + 3.05

### Match meetings to the file of neurologists

In [436]:
usmeetings = meetings.loc[meetings.us==1] 
meetingAMAMatchIndex = recordlinkage.index.SortedNeighbourhood(left_on='lastName', right_on='MailLastName', window=11)
personMatch = meetingAMAMatchIndex.index(usmeetings, allNeuro)
personMatch.names = ['meetingID', 'amaID']
dfTest = pd.DataFrame(index=personMatch)
allNeuro.index.name = 'amaID'

In [437]:
personComp = recordlinkage.Compare(n_jobs=12)
personComp.string('lastName', 'MailLastName', method='jarowinkler')
personComp.string('firstName', 'MailFirstName', method='jarowinkler')
personComp.string('middleInitial', 'middleInitial', method='jarowinkler')
personComp.exact('female', 'female')
personComp.numeric('medGradYear', 'YOGEst', method='linear', offset=2, scale=2)

personCompared = personComp.compute(personMatch, usmeetings, allNeuro)
personCompared = personCompared.rename({0:"lastNameWeight", 1:"firstNameWeight", 2:"middleInitialWeight", 3:"femaleWeight", 4:"medGradWeight"}, axis='columns')
personCompared['totalWeight'] = personCompared.lastNameWeight + personCompared.firstNameWeight + personCompared.middleInitialWeight + personCompared.femaleWeight + personCompared.medGradWeight
personCompared['arbWeight'] = personCompared.lastNameWeight + personCompared.firstNameWeight * 0.8 + personCompared.middleInitialWeight * 0.25 + personCompared.femaleWeight + personCompared.medGradWeight * 0.25

In [438]:
joined = personCompared.join(usmeetings, on='meetingID', how='left').join(allNeuro, on='amaID', how='left', lsuffix='left', rsuffix='right')

In [439]:
joined['percentWeight'] = joined.totalWeight / joined.totalNonMissing
joined['arbPercentWeight'] = joined.arbWeight / joined.weightedNonMissing
joined.sort_values(['meetingID', 'percentWeight', 'arbPercentWeight'], ascending=False, inplace=True)
simpleJoin = joined[[ 'percentWeight', 'arbPercentWeight', 'lastName', 'firstName', 'femaleleft', 'middleInitialleft', 'medGradYear', 'MailFirstName', 'MailLastName', 'middleInitialright', 'femaleright', 'YOGEst']]


### Multiple manual comparisons bewteen matches using the (commented out to make notebook more succinct) code below, generating the file matchData.xlsx

In [440]:
#simpleJoin.loc[(1490,)]

In [441]:
#joined.loc[(1490,)]

In [442]:
joined['rankWithinMeeting'] = joined.groupby(["meetingID"]).cumcount()
joined['nextPercentWeight'] = joined.groupby('meetingID')['percentWeight'].shift(-1) 
joined['nextPercentArb'] = joined.groupby('meetingID')['arbPercentWeight'].shift(-1) 
joined['weightDelta'] = joined.percentWeight - joined.nextPercentWeight 
joined['arbDelta'] = joined.arbPercentWeight - joined.nextPercentArb
joined['exactNameMatch'] = (joined.lastNameWeight==1) & (joined.firstNameWeight==1)

In [443]:
exactMatchesForMeeting = joined.groupby("meetingID").exactNameMatch.sum()
exactMatchesForMeeting.rename("exactMatchesForMeeting", inplace=True)
joined = joined.join(other=exactMatchesForMeeting, on='meetingID', how='left')

In [444]:
bestMatchForMeeting = joined.loc[joined['rankWithinMeeting'] == 0]

In [445]:
matchCharacteristics[['arb','match', 'weight', 'weightDelta', 'arbDelta']].groupby(['match']).describe()

arb                                                  arbDelta  \
      count      mean       std   min   25%   50%  75%   max    count   
match                                                                   
0      41.0  0.743902  0.092517  0.53  0.71  0.76  0.8  0.91     12.0   
1      19.0  0.982632  0.039978  0.83  0.98  1.00  1.0  1.00     18.0   

                                                             weight            \
           mean       std   min     25%    50%     75%   max  count      mean   
match                                                                           
0      0.169167  0.096527 -0.01  0.1275  0.190  0.2225  0.32   41.0  0.669756   
1      0.308889  0.092284  0.10  0.2725  0.315  0.3700  0.47   19.0  0.963684   

                                              weightDelta                      \
            std   min   25%   50%   75%   max       count      mean       std   
match                                                                           
0      0.124127  0.42  0.61  0.64  0.75  0.89        12.0  0.105833  0.094239   
1      0.069138  0.73  0.94  1.00  1.00  1.00        18.0  0.294444  0.100131   

                                          
        min     25%    50%     75%   max  
match                                     
0      0.00  0.0075  0.115  0.1675  0.27  
1      0.06  0.2500  0.300  0.3700  0.47

1. The highest unmathced weight is 0.89 and the highest unmathced arb is 0.91...so, we can use those as simple threshods for "if you're above this...call it a match"

2. Matches also have higher "gaps" between themselves and the next best match. so, we can probably also build a criterion on a large "gap" and a slightly lowe weight. Maybe go down to the lowest matched weight (0.73) and arb (0.83) with a 25th percentile matched delta (arb 0.27, weight 0.25) which is larger than the 75th percentile for non matches.

### make matches. in neuro


In [446]:
nearExactWeightMatches = bestMatchForMeeting.loc[bestMatchForMeeting.percentWeight > 0.90]
nearExactArbWeightMatches = bestMatchForMeeting.loc[bestMatchForMeeting.arbPercentWeight > 0.92]

moderateWeightMatchWithLargeGap = bestMatchForMeeting.loc[(bestMatchForMeeting.percentWeight <= 0.90) & (bestMatchForMeeting.percentWeight > 0.73) & (bestMatchForMeeting.weightDelta > 0.25)]
moderateArbMatchWithLargeGap = bestMatchForMeeting.loc[(bestMatchForMeeting.arbPercentWeight <= 0.92) & (bestMatchForMeeting.arbPercentWeight > 0.83) & (bestMatchForMeeting.arbDelta > 0.27)]

exactAndOnlyNameMatch = bestMatchForMeeting.loc[(bestMatchForMeeting.exactNameMatch) & (bestMatchForMeeting.exactMatchesForMeeting==1)]

neuroMatches = pd.concat([nearExactArbWeightMatches, nearExactWeightMatches, moderateArbMatchWithLargeGap, moderateWeightMatchWithLargeGap, exactAndOnlyNameMatch]).drop_duplicates()

60% seem are matched using pretty solid near exact matches. will 

In [447]:
len(neuroMatches)/len(usmeetings)

0.6259228876127974

### now try to match the unmatched to the non-neuro ama file

In [448]:
unmatchedMeetings = usmeetings.loc[~usmeetings.index.isin(neuroMatches.index.get_level_values(0))]
nonNeuro = ama.loc[ama.ResearchID.isin(set(ama.ResearchID) - set(allNeuro.ResearchID))]

In [449]:
nonNeuroMeetingAMAMatch = recordlinkage.index.SortedNeighbourhood(left_on='lastName', right_on='MailLastName', window=11)
nonNeuroPersonMatch = nonNeuroMeetingAMAMatch.index(unmatchedMeetings, nonNeuro)
nonNeuroPersonMatch.names = ['meetingID', 'amaID']

In [450]:
nonNeuroComp = recordlinkage.Compare(n_jobs=12)
nonNeuroComp.string('lastName', 'MailLastName', method='jarowinkler')
nonNeuroComp.string('firstName', 'MailFirstName', method='jarowinkler')
nonNeuroComp.string('middleInitial', 'middleInitial', method='jarowinkler')
nonNeuroComp.exact('female', 'female')
nonNeuroComp.numeric('medGradYear', 'YOGEst', method='linear', offset=2, scale=2)

nonNeuroCompared = nonNeuroComp.compute(nonNeuroPersonMatch, unmatchedMeetings, nonNeuro)

In [451]:
nonNeuroCompared = nonNeuroCompared.rename({0:"lastNameWeight", 1:"firstNameWeight", 2:"middleInitialWeight", 3:"femaleWeight", 4:"medGradWeight"}, axis='columns')
nonNeuroCompared['totalWeight'] = nonNeuroCompared.lastNameWeight + nonNeuroCompared.firstNameWeight + nonNeuroCompared.middleInitialWeight + nonNeuroCompared.femaleWeight + nonNeuroCompared.medGradWeight
nonNeuroCompared['arbWeight'] = nonNeuroCompared.lastNameWeight + nonNeuroCompared.firstNameWeight * 0.8 + nonNeuroCompared.middleInitialWeight * 0.25 + nonNeuroCompared.femaleWeight + nonNeuroCompared.medGradWeight * 0.25

In [452]:
nonNeuroJoined = nonNeuroCompared.join(unmatchedMeetings, on='meetingID', how='left').join(nonNeuro, on='amaID', how='left', lsuffix='left', rsuffix='right')

nonNeuroJoined['percentWeight'] = nonNeuroJoined.totalWeight / nonNeuroJoined.totalNonMissing
nonNeuroJoined['arbPercentWeight'] = nonNeuroJoined.arbWeight / nonNeuroJoined.weightedNonMissing

nonNeuroJoined.sort_values(['meetingID', 'percentWeight', 'arbPercentWeight'], ascending=False, inplace=True)
nonNeuroSimpleJoin = nonNeuroJoined[[ 'percentWeight', 'arbPercentWeight', 'lastName', 'firstName', 'femaleleft', 'middleInitialleft', 'medGradYear', 'MailFirstName', 'MailLastName', 'middleInitialright', 'femaleright', 'YOGEst']]

In [453]:
nonNeuroSimpleJoin.index.get_level_values(0).unique()

Int64Index([1492, 1490, 1489, 1488, 1487, 1486, 1484, 1480, 1476, 1474,
            ...
              61,   48,   47,   45,   44,   36,   20,   19,   16,   10],
           dtype='int64', name='meetingID', length=456)

In [454]:
#nonNeuroSimpleJoin.loc[(44,)]

In [455]:
nonNeuroJoined.sort_values(['meetingID', 'arbPercentWeight', 'percentWeight'], ascending=False, inplace=True)
nonNeuroJoined['rankWithinMeeting'] = nonNeuroJoined.groupby(["meetingID"]).cumcount()
nonNeuroJoined['nextPercentWeight'] = nonNeuroJoined.groupby('meetingID')['percentWeight'].shift(-1) 
nonNeuroJoined['nextPercentArb'] = nonNeuroJoined.groupby('meetingID')['arbPercentWeight'].shift(-1) 
nonNeuroJoined['weightDelta'] = nonNeuroJoined.percentWeight - nonNeuroJoined.nextPercentWeight 
nonNeuroJoined['arbDelta'] = nonNeuroJoined.arbPercentWeight - nonNeuroJoined.nextPercentArb

bestNonNeuroMatchForMeeting = nonNeuroJoined.loc[nonNeuroJoined['rankWithinMeeting'] == 0]

In [456]:
nonNeuroNearExactWeightMatches = bestNonNeuroMatchForMeeting.loc[(bestNonNeuroMatchForMeeting.percentWeight > 0.90) & (bestNonNeuroMatchForMeeting.weightDelta > 0.01)]
nonNeuroNearExactArbWeightMatches = bestNonNeuroMatchForMeeting.loc[(bestNonNeuroMatchForMeeting.arbPercentWeight > 0.92) & (bestNonNeuroMatchForMeeting.arbDelta > 0.01)]

nonNeuroModerateWeightMatchWithLargeGap = bestNonNeuroMatchForMeeting.loc[(bestNonNeuroMatchForMeeting.percentWeight <= 0.90) & (bestNonNeuroMatchForMeeting.percentWeight > 0.73) & (bestNonNeuroMatchForMeeting.weightDelta > 0.25)]
nonNeuroModerateArbMatchWithLargeGap = bestNonNeuroMatchForMeeting.loc[(bestNonNeuroMatchForMeeting.arbPercentWeight <= 0.92) & (bestNonNeuroMatchForMeeting.arbPercentWeight > 0.83) & (bestNonNeuroMatchForMeeting.arbDelta > 0.27)]

nonNeuroMatches = pd.concat([nonNeuroNearExactWeightMatches, nonNeuroNearExactArbWeightMatches, nonNeuroModerateWeightMatchWithLargeGap, nonNeuroModerateArbMatchWithLargeGap]).drop_duplicates()

In [457]:
len(nonNeuroMatches)/len(usmeetings)

0.2026251025430681

In [460]:
unmatchedMeetings2 = unmatchedMeetings.loc[~unmatchedMeetings.index.isin(nonNeuroMatches.index.get_level_values(0))]
len(unmatchedMeetings2)/len(usmeetings)

0.17145200984413453

In [459]:
#unmatchedMeetings2.tail(50)

### factors we can match on directly: 
<ol>
    <li>first name</li>
    <li>last name</li>
    <li>middle initial</li>
    <li> sex </li>
    <li>med school year of graduation</li>
</ol>

### factors that we can match/filter on somewhat: 
<ol>
    <li> residency training institution vs. current institution (assuming there is a lot of overlap)</li>
    <li> curent hospital name...althjough huge limitation that its missing for 85% of the samle</li>
    <li> current mailing zip code (ama masterfile) vs. hospital zip code (will n eed to be matched into speaker file</li>
    <li>US trained in both files</li>
    <li> maybe there is a way to link the academic filter? although probably not that useful given its prevalence</li>
</ol>

    

In [500]:
meetings['neuroMatch'] = meetings.index.isin(neuroMatches.index.get_level_values(0))
meetings['nonNeuroMatch'] =meetings.index.isin(nonNeuroMatches.index.get_level_values(0))
meetings['unmatched'] = meetings.index.isin(unmatchedMeetings2.index)
meetings[['neuroMatch','nonNeuroMatch', 'unmatched', 'nonus']] = meetings[['neuroMatch','nonNeuroMatch', 'unmatched', 'nonus']].astype(int)
meetings['matchCategory'] = meetings[['neuroMatch','nonNeuroMatch', 'unmatched', 'nonus']].idxmax(axis=1)

### How was the overall match?

In [518]:
meetings.matchCategory.value_counts()

neuroMatch       763
nonus            275
nonNeuroMatch    247
unmatched        209
Name: matchCategory, dtype: int64

### Compare the speakers data across the categories of matches...

1. We have a small problem with the unmatched group where women are more common
2. Women speakers are less common amongst non-US speakers...
3. Non-US speakers have higher pub counts
4. non-neuro meetings have more recnet grads with fewer pubs

In [521]:
meetings.replace(-1, np.nan, inplace=True)
meetings.pivot_table(index=meetings.matchCategory, dropna=True, aggfunc=( 'mean', 'std')).transpose()

matchCategory             neuroMatch  nonNeuroMatch        nonus    unmatched
academic           mean     0.955439       0.947368     0.989091     0.937799
                   std      0.206473       0.223750     0.104065     0.242100
female             mean     0.284404       0.303644     0.185654     0.364078
                   std      0.451426       0.460764     0.389650     0.482343
honorificSpeaker   mean     0.000000       0.000000     0.000000     0.000000
                   std      0.000000       0.000000     0.000000     0.000000
leadership         mean     0.369594       0.141700     0.047273     0.110048
                   std      0.709677       0.502192     0.272763     0.395096
medGradYear        mean  1992.120211    1995.636735  1992.920455  1992.896552
                   std     10.935360      10.582360     9.135282    11.422556
neuroMatch         mean     1.000000       0.000000     0.000000     0.000000
                   std      0.000000       0.000000     0.000000     0.000000
nonNeuroMatch      mean     0.000000       1.000000     0.000000     0.000000
                   std      0.000000       0.000000     0.000000     0.000000
nonus              mean     0.000000       0.000000     1.000000     0.000000
                   std      0.000000       0.000000     0.000000     0.000000
pubsScopus         mean   121.025066      95.107438   172.514894   104.636816
                   std    134.809952     108.450520   161.102514   145.488083
scopusHIndex       mean    29.268519      24.454545    32.773504    24.879397
                   std     26.700857      21.924238    23.104446    25.750464
speaker            mean     0.722149       0.331984     0.105455     0.287081
                   std      1.147344       0.700689     0.467724     1.062415
totalNonMissing    mean     4.515072       4.246964     4.192727     4.244019
                   std      0.500101       0.432121     0.395160     0.430535
unmatched          mean     0.000000       0.000000     0.000000     1.000000
                   std      0.000000       0.000000     0.000000     0.000000
us                 mean     1.000000       1.000000     0.000000     1.000000
                   std      0.000000       0.000000     0.000000     0.000000
weightedNonMissing mean     3.178768       3.111741     3.098182     3.111005
                   std      0.125025       0.108030     0.098790     0.107634